In [5]:
import os

In [7]:
# since this notebook is inside research folder, this notebook automatically executed in that folder.
# we can test this using this command and it will show us which directory we're in
%pwd
# we need to change to work inside chicken-disease-classification-project (we go back one level)

'c:\\Users\\ben jazia\\OneDrive\\Documents\\GitHub\\Chicken-Disease-MLOps project\\Chicken-Disease-Classification-Project'

In [6]:
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path
# this is an entity and it will be copied inside config_entity.py
# @dataclass → a decorator that automatically gives you a constructor (__init__), __repr__, __eq__, etc.
# frozen=True → makes the object immutable (once created, you can’t change its fields)
# which is useful for configs, we dont want them to be changed by mistake
@dataclass(frozen=True)
class DataIngestionConfig: # = Entity. a structured object that holds all the config values for the ingestion step:
    root_dir: Path # where artifacts are stored (like artifacts/data_ingestion)
    source_URL: str # dataset link (my S3 URL)
    local_data_file: Path # path to store the downloaded zip
    unzip_dir: Path # path where the zip is extracted

# The role of Entities here:
# Instead of passing around raw dictionaries or YAML everywhere, you load the YAML once and convert it into this Entity.
# Entities = structured data holders. They map configs (from YAML) into Python objects with type safety.
# Entities act as bridges between your config.yaml file and the pipeline code:
# You load YAML.
# You wrap values into Entities (DataIngestionConfig, …).
# You pass those Entities to the pipeline components.

In [10]:
from cnnChickenDiseaseClassifier.constants import *
from cnnChickenDiseaseClassifier.utils.common import read_yaml, create_directories

### ^ Why can we type cnnChickenDiseaseClassifier.constants instead of a real path ?
because that's how packages work. since there's an __init__.py file in cnnChickenDiseaseClassifier, it's treated as a package. 
When I installed my project in editable mode (pip install -e .), Python adds my src/ folder to the environment’s import path (sys.path).
which is why we can use a dot (because import system uses dots) instead of C:/..

## Now, we're making the ConfigurationManager class.
Think of it as the brain that manages configs during runtime:
- Converts raw YAML values into a strongly-typed object (DataIngestionConfig) that can be passed to components like DataIngestion.
- This makes downstream code cleaner: no dict lookups everywhere, just attributes (config.root_dir etc.).

#### ConfigurationManager should be inside config/configuration.py in most cases,
but here I think the tutorial is just using the notebook to experiment and explain before moving it there.

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # = "create_directories(["artifacts"])""
        # ^ we can use the . here instead of [artifacts] because, remember, read_yaml returns a Box Type (this is exactly why we made it return ConfigBox)


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: # here, we're making it return a DataIngestionConfig type, which we defined earlier as an Entity (structure) for data ingestion configuration
        config = self.config.data_ingestion 

        create_directories([config.root_dir]) # will create artifacts/data_ingestion as specified in config.yaml

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from cnnChickenDiseaseClassifier import logger
from cnnChickenDiseaseClassifier.utils.common import get_size

### The DataIngestion class
This class is the component that actually performs ingestion. We will copy it isnide components/data_ingestion.py


It expects a DataIngestionConfig object to know where to download, where to store the zip, where to unzip, etc.

#### How they work together
ConfigurationManager loads YAML → creates DataIngestionConfig.
DataIngestion takes that config → runs logic (download + unzip).

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
# Idempotent download: if data.zip isn’t there, download it; otherwise log size and skip.
# urlretrieve returns (filename, headers); they log both.
# Uses a helper get_size(...) (we have it in utils/common.py) and a logger.


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    # Ensures the unzip directory exists, then extracts the zip there.

### Now, we create the pipeline

this little block is the “glue” that ties together everything we built

In [ ]:
try: # This code will be copied in the data_ingestion pipeline inside main in stage_01_data_ingestion
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-09 21:11:29,413: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-09 21:11:29,415: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-09 21:11:29,417: INFO: common: created directory at: artifacts]
[2025-09-09 21:11:29,419: INFO: common: created directory at: artifacts/data_ingestion]


[2025-09-09 21:28:59,593: INFO: 546546027: artifacts/data_ingestion/data.zip download! with following info: 
x-amz-id-2: G0Ax7ylopw7x4tsbybt+5vnx8/5fPT0sMn1Cycta12TcQmXHju/NNkz1GOAz/jLZ5PJeM2ZV4wg=
x-amz-request-id: 3QDH6ZAZAYGBSDR7
Date: Tue, 09 Sep 2025 20:11:36 GMT
Last-Modified: Tue, 09 Sep 2025 18:34:34 GMT
ETag: "bb6a3a82d4f888efb31eb0698bed0fc9"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Content-Length: 11616915
Server: AmazonS3
Connection: close

]
